# **Jupyter notebook front-end (drop-in cells)**


**Smoke tests (copy/paste)**

**Run these whenever something feels off.**

**A. Confirm the router is mounted**

In [20]:
import os
os.chdir(r"C:\Users\User\PortaeOS-skeleton\packages\agenticcore")  # <-- adjust to your repo root

# Python one-liner in the same env where the server runs
import sys; sys.path.insert(0,'.')
import backend.app.main as m
[(getattr(r,'path',None), getattr(r,'methods',None)) for r in m.app.routes]

# Expect to see ('/chatbot/message', {'POST'}) in the list


[ui] serving SPA from: C:\Users\User\PortaeOS-skeleton\packages\shell\dist


[('/openapi.json', {'GET', 'HEAD'}),
 ('/docs', {'GET', 'HEAD'}),
 ('/docs/oauth2-redirect', {'GET', 'HEAD'}),
 ('/redoc', {'GET', 'HEAD'}),
 ('/health', {'GET'}),
 ('/status', {'GET'}),
 ('/services', {'GET'}),
 ('/services/{name}/start', {'POST'}),
 ('/services/{name}/stop', {'POST'}),
 ('/services/{name}/restart', {'POST'}),
 ('/logs/{service}/tail', {'GET'}),
 ('/logs/bundle.zip', {'GET'}),
 ('/favorites', {'GET'}),
 ('/macros/open', {'POST'}),
 ('/license/validate', {'POST'}),
 ('/agents/run', {'POST'}),
 ('/terminals', None),
 ('/storefront/postman-collection', {'GET'}),
 ('/office/status', {'GET'}),
 ('/office/run-macro', {'POST'}),
 ('/ui', None),
 ('/', {'GET'}),
 ('/ui/{_:path}', {'GET'}),
 ('/ai/ping', {'GET'}),
 ('/ai/health', {'GET'}),
 ('/ai/agents/dispatch', {'POST'}),
 ('/ai/ingest', {'POST'}),
 ('/ai/search', {'GET'}),
 ('/ai/chat', {'POST'}),
 ('/recode/health', {'GET'}),
 ('/terminals/{tid}', None),
 ('/terminals/spawn/{tid}', {'GET'}),
 ('/terminals/kill/{tid}', {'G

(You already did a version of this and saw /chatbot/message appear — perfect.)

**B. Health → Chat via API**

In [6]:
import requests, json, os
BASE = os.environ.get("BACKEND_URL","http://127.0.0.1:8000").rstrip("/")
print("Health:", requests.get(BASE+"/health").json())
r = requests.post(BASE+"/chatbot/message", json={"message":"hello via api"})
print("Reply:", r.status_code, r.json())


Health: {'ok': True, 'version': '0.3.0', 'time': 1757812216}
Reply: 200 {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5, 'thread': None}


**C. Library path (no server required)**

In [7]:
from agenticcore.chatbot.services import ChatBot
print(ChatBot().reply("hello via library"))


{'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5}


# **Minimal notebook “front-end” cells (drop into top of your .ipynb)**

These mirror your working UI and give you pass/fail signals inside the notebook.

In [8]:
# Cell 1: config + helpers
import os, json, requests
BACKEND_URL = os.environ.get("BACKEND_URL", "http://127.0.0.1:8000").rstrip("/")

def health(url: str = BACKEND_URL): 
    r = requests.get(url + "/health", timeout=10); r.raise_for_status(); return r.json()

def send_via_api(message: str, url: str = BACKEND_URL):
    r = requests.post(url + "/chatbot/message", json={"message": message}, timeout=20)
    r.raise_for_status(); return r.json()

def send_via_library(message: str):
    from agenticcore.chatbot.services import ChatBot
    return ChatBot().reply(message)

print("BACKEND_URL =", BACKEND_URL)
print("Health:", health())


BACKEND_URL = http://127.0.0.1:8000
Health: {'ok': True, 'version': '0.3.0', 'time': 1757812221}


In [9]:
# Cell 2: quick acceptance checks
lib = send_via_library("hello")
assert all(k in lib for k in ("reply","sentiment","confidence"))
print("Library OK:", lib)

api = send_via_api("hello from api")
assert all(k in api for k in ("reply","sentiment","confidence"))
print("API OK:", api)


Library OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5}
API OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5, 'thread': None}


**Cell 1 — Config & helpers**

In [10]:
# Notebook Config
import os, json, requests
from typing import Dict, Any

BACKEND_URL = os.environ.get("BACKEND_URL", "http://127.0.0.1:8000").rstrip("/")

def health(url: str = BACKEND_URL) -> Dict[str, Any]:
    """GET /health to verify server is up."""
    r = requests.get(url + "/health", timeout=10)
    r.raise_for_status()
    return r.json()

def send_via_api(message: str, url: str = BACKEND_URL) -> Dict[str, Any]:
    """POST to FastAPI /chatbot/message. Returns reply/sentiment/confidence."""
    r = requests.post(url + "/chatbot/message", json={"message": message}, timeout=20)
    r.raise_for_status()
    return r.json()

def send_via_library(message: str) -> Dict[str, Any]:
    """Call ChatBot() directly (no server needed)."""
    from agenticcore.chatbot.services import ChatBot
    return ChatBot().reply(message)

print("BACKEND_URL =", BACKEND_URL)


BACKEND_URL = http://127.0.0.1:8000


**Cell 2 — Widget UI (switch API / Library)**

In [11]:
import ipywidgets as W
from IPython.display import display, HTML

mode = W.ToggleButtons(options=[("API", "api"), ("Library", "lib")], value="api", description="Route:")
backend = W.Text(value=BACKEND_URL, description="Backend:", layout=W.Layout(width="60%"))
save_btn = W.Button(description="Save", button_style="info")
msg = W.Text(placeholder="Type a message…", description="You:", layout=W.Layout(width="60%"))
send_btn = W.Button(description="Send", button_style="primary")
cap_btn = W.Button(description="Capabilities")
out = W.Output()

def on_save(_):
    os.environ["BACKEND_URL"] = backend.value.strip().rstrip("/")
    with out: print("[config] BACKEND_URL =", os.environ["BACKEND_URL"])

def on_send(_):
    text = msg.value.strip()
    if not text:
        with out: print("[warn] Please enter some text.")
        return
    try:
        data = send_via_api(text, backend.value.strip()) if mode.value == "api" else send_via_library(text)
        with out: print(json.dumps(data, indent=2, ensure_ascii=False))
    except Exception as e:
        with out: print(f"[error] {e}")

def on_caps(_):
    try:
        from agenticcore.chatbot.services import ChatBot
        with out: print(json.dumps({"capabilities": ChatBot().capabilities()}, indent=2))
    except Exception as e:
        with out: print(f"[error capabilities] {e}")

save_btn.on_click(on_save); send_btn.on_click(on_send); cap_btn.on_click(on_caps)

display(W.HBox([mode, backend, save_btn]))
display(W.HBox([msg, send_btn, cap_btn]))
display(out)
display(HTML('<div style="margin-top:8px;opacity:.8">Tip: ensure FastAPI exposes <code>/chatbot/message</code>. Switch to Library for offline tests.</div>'))


Output()

**Cell 3 — Smoke checks (acceptance)**

In [12]:
# Backend health (if running)
try:
    print("Health:", health(backend.value.strip()))
except Exception as e:
    print("Health check failed:", e)

# Library path always available
sample = send_via_library("hello")
assert all(k in sample for k in ("reply", "sentiment", "confidence"))
print("Library OK:", sample)

# API path (requires uvicorn backend running)
try:
    sample_api = send_via_api("hello from api", backend.value.strip())
    assert all(k in sample_api for k in ("reply", "sentiment", "confidence"))
    print("API OK:", sample_api)
except Exception as e:
    print("API test failed (start uvicorn?):", e)


Health: {'ok': True, 'version': '0.3.0', 'time': 1757812228}
Library OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5}
API OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5, 'thread': None}


**Cell 4 — Minimal report cell (optional screenshots prompt)**

In [13]:
from IPython.display import Markdown
Markdown("""
### What to capture for the report
- Screenshot of **/health** and a successful **/chatbot/message** call.
- Notebook output using **API** mode and **Library** mode.
- Short note: environment variables used (e.g., `MICROSOFT_AI_*`, `AI_PROVIDER`, `HF_API_KEY`).
- Brief discussion of any errors and fixes (e.g., route mounting, ports).
""")



### What to capture for the report
- Screenshot of **/health** and a successful **/chatbot/message** call.
- Notebook output using **API** mode and **Library** mode.
- Short note: environment variables used (e.g., `MICROSOFT_AI_*`, `AI_PROVIDER`, `HF_API_KEY`).
- Brief discussion of any errors and fixes (e.g., route mounting, ports).
